<a href="https://colab.research.google.com/github/nullvoid-j/Genre_Predection/blob/master/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#mounting google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import re
from os import listdir
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import numpy as np
import pandas as pd
import collections

In [0]:
#funtion to clean data
def clean_data(data):
    data=re.sub(r"\n", " ", data)   
    data=re.sub(r"[^a-zA-Z0-9]", " ", data)   
    data=re.sub(r"\s\s+", " ", data)   
    data=data.lower() 
    return data

In [0]:
files=listdir('/content/drive/My Drive/Movie Scripts data/')
len(files)

2837

In [0]:
# print([item for item, count in collections.Counter(a).items() if count>1])

[1, 2, 5]


In [0]:
list_of_docs=[]
for file in files:
  filtered_files=[]
  file_data=open('/content/drive/My Drive/Movie Scripts data/'+file, 'r')
  data=clean_data(file_data.read())
  for w in data.split():
    if w.isalpha():
      filtered_files.append(w)
    else:
      continue
  list_of_docs.append(filtered_files)
  file_data.close()
  print(file+' has been processed')

In [0]:
len(list_of_docs)

2837

In [0]:
# #training own w2v model
# W2V_model=Word2Vec(list_of_docs, min_count=5, size=200, workers=4, seed=1)
# W2V_model.save('self_trained_w2v.model')

# words=list(W2V_model.wv.vocab)
# print(len(words))

In [0]:
#getting pre trained embedding file
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-04-05 12:26:40--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.94.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.94.157|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
#using google trained model
w2v_google_model=KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#now computing average vector for each script
doc_vec_list=[]
for doc in list_of_docs:
  doc_vec=np.zeros(300)
  no_of_words=0
  for word in doc:
    try:
      word_vec=w2v_google_model.wv[word]
      doc_vec += word_vec
      no_of_words += 1
    except:
      # print("There was an exception. Passing...")
      pass
  doc_vec=doc_vec/no_of_words
  doc_vec_list.append(doc_vec)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [0]:
len(doc_vec_list)

2837

In [0]:
vec_df=pd.DataFrame(data=doc_vec_list)
file_name_df=pd.DataFrame(data=files)
vec_df=vec_df.join(file_name_df, rsuffix='file')

In [0]:
vec_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,0file
0,0.018198,0.044626,0.029221,0.114274,-0.040054,-0.018154,-0.000469,-0.082588,0.051520,0.055732,-0.045181,-0.114221,-0.054481,0.009727,-0.097010,0.061535,0.053665,0.102929,0.011613,-0.061160,-0.037943,0.040869,0.068884,-0.007697,0.007207,-0.010008,-0.089893,0.060147,0.022216,-0.013505,0.005475,0.046188,-0.040408,-0.018359,-0.044276,-0.005771,-0.010326,0.043276,0.025405,0.068651,...,-0.003131,-0.038233,0.001674,0.032924,0.094447,-0.023062,-0.011947,-0.048910,-0.018729,-0.009933,0.068761,0.082313,0.039279,0.065177,-0.054247,-0.062719,-0.103622,-0.059352,0.028182,0.002008,0.000323,0.029794,0.063573,0.015085,0.013972,-0.053627,-0.021785,0.003510,0.015722,-0.018123,0.045903,-0.104959,0.022389,-0.053999,-0.041609,-0.005899,-0.055441,-0.006187,0.019514,file_2644.txt
1,0.051512,0.040707,0.031853,0.077093,-0.050945,-0.030842,0.028621,-0.089511,0.066415,0.076311,-0.039354,-0.110417,-0.020285,0.014459,-0.101189,0.049245,0.044111,0.095099,0.015709,-0.051280,-0.017178,0.063895,0.016417,-0.015360,0.037992,-0.040598,-0.076229,0.058830,0.040326,-0.004434,-0.020760,0.022772,-0.055106,0.000795,-0.003474,-0.031029,0.010205,0.008735,0.021876,0.058820,...,-0.005563,-0.054537,0.025243,0.053733,0.120033,-0.010281,0.004869,-0.049528,-0.011786,0.024220,0.083910,0.080570,0.027076,0.060135,-0.055828,-0.054877,-0.106980,-0.034053,-0.009890,0.015696,-0.023937,0.016536,0.064059,0.032072,0.014817,-0.066752,-0.008264,0.005825,0.034637,-0.063916,0.022181,-0.094670,0.011859,-0.037805,-0.003549,0.009859,-0.044397,0.020011,-0.006038,file_2645.txt
2,0.037229,0.040462,0.022106,0.097785,-0.063829,-0.023708,0.010958,-0.087613,0.068865,0.049839,-0.050469,-0.131115,-0.045793,-0.000968,-0.098539,0.069786,0.054320,0.100287,0.019829,-0.064366,-0.029129,0.052875,0.047220,-0.021702,0.014436,-0.009913,-0.097177,0.050688,0.025023,0.000357,0.004987,0.026958,-0.054380,-0.004398,-0.009532,-0.014022,0.004467,0.030410,0.032064,0.057278,...,-0.007790,-0.039488,0.024666,0.037175,0.114121,-0.004061,-0.005617,-0.051336,-0.024506,0.029563,0.065577,0.088678,0.053199,0.042148,-0.063362,-0.054529,-0.105451,-0.054931,0.010877,0.003225,-0.006543,0.024976,0.070356,0.020844,0.023846,-0.047246,-0.017209,0.013829,0.034229,-0.056800,0.041335,-0.094746,0.025605,-0.050767,-0.027574,-0.000033,-0.067148,0.009308,0.000077,file_2639.txt
3,0.018072,0.039623,0.014473,0.112143,-0.064708,-0.024511,0.001364,-0.097545,0.056539,0.055179,-0.054712,-0.136767,-0.042074,-0.004972,-0.104225,0.066391,0.051804,0.112859,0.034542,-0.056317,-0.038140,0.047600,0.074735,-0.025814,0.015948,-0.004459,-0.103277,0.061238,0.037522,-0.015130,-0.013890,0.039557,-0.049308,-0.030748,-0.034159,-0.019955,0.010200,0.046962,0.025266,0.060281,...,-0.017011,-0.041929,0.008012,0.032387,0.100329,-0.016538,-0.001011,-0.030402,-0.031034,0.002280,0.086845,0.103354,0.063742,0.056119,-0.057853,-0.050151,-0.123249,-0.053748,0.030366,0.010639,-0.003666,0.032894,0.073165,0.015585,0.008754,-0.053284,-0.040355,0.011283,0.033266,-0.028795,0.053575,-0.082977,0.024344,-0.069255,-0.048858,0.015715,-0.057526,-0.004622,0.013169,file_2647.txt
4,0.021606,0.023598,0.014492,0.094708,-0.042444,-0.027072,0.018890,-0.080777,0.051512,0.052879,-0.062061,-0.122351,-0.034222,-0.011039,-0.117139,0.073524,0.068160,0.116556,0.030134,-0.084146,-0.023731,0.066918,0.089119,-0.023544,-0.003226,-0.027087,-0.090558,0.062889,0.034470,-0.010231,-0.018225,0.054206,-0.048264,-0.020535,-0.031102,-0.013298,0.010180,0.051835,0.025862,0.073322,...,-0.010276,-0.036820,0.001035,0.028786,0.113785,-0.026366,-0.005381,-0.063397,-0.032146,-0.019638,0.077253,0.078259,0.050232,0.049153,-0.051166,-0.045215,-0.110867,-0.068711,0.017452,0.025007,-0.016007,0.038154,0.063641,0.021429,-0.010459,-0.029507,-0.025063,0.0091

In [0]:
train_df=pd.read_csv('/content/drive/My Drive/Movie_test_train_data/Train.csv')
train_df=pd.merge(train_df, vec_df, how='inner', left_on='File_Name',right_on='0file')
train_df.drop(columns='0file', axis=1, inplace=True)

In [0]:
#saving the vectorized form in pickle file
with open('/content/drive/My Drive/W2V_data.pkl','wb') as f:
  pickle.dump(train_df, f)


In [0]:
###################################### Simple model ###################################### 

In [0]:
X=train_df.iloc[:, 2:].values
Y=train_df.iloc[:, 1].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier().fit(x_train, y_train)

In [0]:
pred=clf.predict(x_test)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print(accuracy_score(y_test, pred))
# print(precision_score(y_test, log_pred, average='weighted'))
# print(recall_score(y_test, log_pred, average='weighted'))

0.05555555555555555
